### This file creates embeddings based on the given text data and saves the model

In [1]:
# importing necessary libraries
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- 1. Load the Document ---
# Make sure your file path is correct
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from dotenv import load_dotenv

# Use a Glob pattern to load all .txt files from the 'content' directory
loader = DirectoryLoader(
    "./content/",
    glob="*.txt",
    loader_cls=TextLoader
)
data = loader.load()

# --- 2. Split the Document (with larger chunk size) ---
# We use a larger chunk size to get more context per document chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(data)
print(f"Total number of documents (chunks): {len(docs)}")

# --- 3. Create Embeddings and Vector Store ---
# The embedding model converts text to numerical vectors
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create the vector store and a retriever that gets more documents
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)
# We set 'k' to 10 to retrieve the top 10 most relevant documents
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# --- 4. Define the RAG Prompt and Language Model ---
# We define a custom prompt to encourage a more detailed answer
template = """
You are a helpful assistant. Use the following context to answer the question.
If you don't know the answer, just say that you don't know. Be very detailed and comprehensive in your answer, providing a thorough summary based on the given context.

Context:
{context}

Question:
{question}

Helpful Answer:"""

prompt = PromptTemplate.from_template(template)
load_dotenv()
# Initialize the Gemini Language Model
# Note: You need to set up your Google API key in Colab secrets
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7, google_api_key=os.environ.get('GOOGLE_API_KEY'))

# --- 5. Build the RAG Chain ---
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


Total number of documents (chunks): 258


/tmp/ipykernel_5877/2312775883.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
E0000 00:00:1758084220.223980    5877 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


### Inferencing with the model

In [6]:
# --- 6. Invoke the Chain ---
# The final result will be a more detailed summary
print("\nGenerating summary...\n")
summary_question = "Compare settlement of 2017-2021 to 2025 what are major changes ?"
result = rag_chain.invoke(summary_question)
print(result)


Generating summary...

Based on the provided text, a direct comparison of the 2017-2021 settlement and a 2025 settlement is not possible.  The document details a settlement from 2017-2021 and then another settlement with a period from 2021-2026. There is no information about a separate settlement in 2025.

However, we can compare the 2017-2021 settlement to the 2021-2026 settlement.  The major differences are:

* **Duration:** The 2017-2021 settlement covered a period of 4 years (48 months), from December 1, 2017, to November 30, 2021.  The 2021-2026 settlement extends for 5 years, from December 1, 2021, to December 31, 2026.

* **Wage Grade Correction and ABP Amount:** The 2021-2026 settlement includes specific clauses (11 and 12) addressing the correction of wage grades for G03 and G04 category permanent workmen to G07, and the correction of their ABP (Annual Benefit Payment) amounts from INR 4,080 to INR 5,818.  These corrections are not explicitly mentioned in the description of t

### SAVE THE MODEL

In [5]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

def save_vector_store(persist_directory="vector_db"):
    """
    Loads documents, creates embeddings, and saves the vector store to a directory.
    """
    print("Saving vector store...")
    
    # 1. Load documents
    loader = DirectoryLoader(
        "./content/",
        glob="*.txt",
        loader_cls=TextLoader
    )
    data = loader.load()

    # 2. Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    docs = text_splitter.split_documents(data)

    # 3. Create embeddings
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # 4. Create and persist the vector store
    # This automatically saves the embeddings to the specified directory
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embedding,
        persist_directory=persist_directory
    )
    vectorstore.persist()
    print(f"Vector store saved to '{persist_directory}'")

# Execute the function to save the vector store
save_vector_store()

Saving vector store...
Vector store saved to 'vector_db'


/tmp/ipykernel_5877/1660703030.py:35: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


### THIS CODE IS FOR INFERENCING AFTER SAVING THE MODEL

In [3]:
# --- ONE-TIME SETUP ---

# Import necessary libraries
import os
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Define the persistence directory for your vector store
PERSIST_DIRECTORY = "vector_db"

# 1. Initialize the embedding model (only once)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Load the persisted vector store (only once)
if not os.path.exists(PERSIST_DIRECTORY):
    print(f"Error: Vector store directory '{PERSIST_DIRECTORY}' not found.")
else:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embedding
    )

# 3. Create the retriever (only once)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 4. Define the RAG prompt (only once)
template = """
You are a helpful assistant. Use the following context to answer the question.
If you don't know the answer, just say that you don't know. Be very detailed and comprehensive in your answer, providing a detailed on the given context.

Context:
{context}

Question:
{input}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

# 5. Initialize the LLM (only once)
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0.7,
    google_api_key=os.environ.get('GOOGLE_API_KEY')
)

# 6. Build the RAG chain (only once)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# --- INFERENCE FUNCTION ---

def inference_with_rag(query):
    """
    Performs a RAG query using the pre-loaded chain.
    """
    print(f"\nQuerying: '{query}'")
    if 'rag_chain' in globals():
        result = rag_chain.invoke({"input": query})
        return result['answer']
    else:
        return "RAG chain not initialized. Check for errors during setup."


E0000 00:00:1758092413.932134   42190 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [4]:
# --- EXAMPLE USAGE (looping for multiple queries) ---

while True:
    user_query = input("\nEnter your query (or type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        print("Exiting chatbot. Goodbye!")
        break
    
    response = inference_with_rag(user_query)
    print("\n-------------------\n")
    print(f"Answer: {response}")
    print("\n-------------------\n")


Querying: 'What is the fixed basked amount for the year 2025'

-------------------

Answer: The provided text gives details of several fixed basket amounts across different years and different agreements.  There's no single "fixed basket amount for the year 2025" that applies universally. The amount depends on which agreement and which component of the fixed basket are being considered.

Here's a breakdown based on the available information:


**Agreement 1 (Page 15):** This section details a staggered payment over two years (2017-2018).  It does not extend to 2025.

**Agreement 2 (Page 13-14):** This agreement covers the period from 01.12.2022 to 31.12.2026.  The total fixed basket amount is INR 9,450. However, this is distributed in a staggered manner:

* 40% (INR 3780) from 01.12.2022 to 30.11.2023
* 40% (INR 3780) from 01.12.2023 to 30.11.2024
* 20% (INR 1890) from 01.12.2024 to 30.11.2025

Therefore, for the year 2025 (01.12.2024 to 30.11.2025), 20% of the total fixed basket amou